# Data Mining Assignment 2 (April 2025)

(0369885) Nuraisyah Dahiyah Binti Hashim

In [ ]:
pip install apyori

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
from google.colab import files
from os.path import exists
from os import environ, system

In [ ]:
uploaded = files.upload()

Saving online_retail_II.csv to online_retail_II.csv


In [ ]:
# ------------------------------
# 4.1 DATATSET OVERVIEW
#-------------------------------

# load the online retail data
data = pd.read_csv('online_retail_II.csv')

# drop columns 'Invoice', 'StockCode', 'Quantity', 'Price' and 'Country'
data = data.drop(columns=['Invoice', 'StockCode', 'Quantity', 'Price', 'Country'])

# display basic dataset info
print("\nBasic Information of the Dataset:")
data.info()

# checking for missing values
print("\nChecking for Missing Values")
print(data.isnull().sum())

# checking for duplicate records
duplicates = data.duplicated().sum()
print(f"\nNumber of Duplicate Records: {duplicates}")

# descriptive statistics to understand item frequencies and transaction distribution
print("\nDescriptive Statistics")
print(data.describe(include='all'))

# display the first few rows
data.head()

# unique items in transactions
print("\nNUmber of Unique Items:", data['Description'].nunique())
print("Number of Unique Transactions:", data['Customer ID'].nunique())


Basic Information of the Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Description  1062989 non-null  object 
 1   InvoiceDate  1067371 non-null  object 
 2   Customer ID  824364 non-null   float64
dtypes: float64(1), object(2)
memory usage: 24.4+ MB

Checking for Missing Values
Description      4382
InvoiceDate         0
Customer ID    243007
dtype: int64

Number of Duplicate Records: 49816

Descriptive Statistics
                               Description          InvoiceDate    Customer ID
count                              1062989              1067371  824364.000000
unique                                5698                47635            NaN
top     WHITE HANGING HEART T-LIGHT HOLDER  2010-12-06 16:57:00            NaN
freq                                  5918                 1350            NaN
mean             

In [ ]:
# ------------------------------
# 4.2 DATATSET PREPROCESSING
#-------------------------------

# renaming columns for consistency and ease of use
data.rename(columns={'Customer ID': 'Transaction', 'Description': 'Item', 'Invoice': 'Date'}, inplace=True)

# handling missing values
data = data.dropna()
print(f"\nAfter dropping missing values, shape of data: {data.shape}")

# handling duplicates
data = data.drop_duplicates()
print(f"\nAfter dropping duplicates, shape of data: {data.shape}")

# removing duplicate rows
data = data.drop_duplicates()
print(f"\nAfter dropping duplicates, shape of data: {data.shape}")

# ----------
# Define suspicious keywords (lowercase)
keywords_to_remove = ['adjustment', 'bank', 'carriage', 'check', 'commission', 'postage', 'manual', 'test', 'dotcom']

# Filter out rows that contain any of the keywords (case-insensitive search)
pattern = '|'.join(keywords_to_remove)
data = data[~data['Item'].str.lower().str.contains(pattern, na=False)]

# Confirm filtering
print("\nRemoved rows containing non-giftware keywords.")
print("Remaining unique descriptions:", data['Item'].nunique())
# ----------
# standardising the Item column for consistency
data['Item'] = data['Item'].astype(str).str.strip().str.lower().astype('category')
data['Transaction']= data['Transaction'].astype(int)

# transforming data into transaction format for the apriori algorithm
transactions = data.groupby('Transaction')['Item'].apply(list).tolist()

# display a sample of preprocessed transaction
print("\nSample Preprocessed Transactions:")
print(transactions[:5])


After dropping missing values, shape of data: (779680, 3)

After dropping duplicates, shape of data: (779680, 3)

After dropping duplicates, shape of data: (779680, 3)

Removed rows containing non-giftware keywords.
Remaining unique descriptions: 5221

Sample Preprocessed Transactions:
[['red spotty childs umbrella', 'edwardian parasol red', 'edwardian parasol natural', 'edwardian parasol black', 'edwardian parasol pink', 'doormat spotty home sweet home', 'doormat welcome to our home', 'doormat i love london', 'doormat christmas village', 'doormat black flock', 'doormat welcome sunrise', 'doormat 3 smiley cats', 'doormat home sweet home blue', 'doormat fancy font home sweet home', 'doormat union flag', 'doormat merry christmas red', 'doormat respectable house', 'doormat fairy cake', 'doormat peace on earth blue', 'doormat welcome puppies', 'doormat new england', 'doormat airmail', 'doormat red spot', 'doormat union jack guns and roses', 'discount', 'medium ceramic top storage jar', 'm

In [1]:
# --------------------------------------------------------
# 4.3 FREQUENT ITEMSETS GENERATION WITH APRIORI ALGORITHM
#---------------------------------------------------------

# Transforming transactions to a one-hot encoded DataFrame
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
data_trans = pd.DataFrame(te_ary, columns=te.columns_)

# display the one-hot encoded data
print('\nOne-Hot Encoded Data:')
print(data_trans.head())

# applying the Apriori algorithm with a low minimum support threshold to get more patterns
frequent_itemsets = apriori(data_trans, min_support=0.01, use_colnames=True)
frequent_itemsets['itemset_length'] = frequent_itemsets['itemsets'].apply(len)

# sorting frequent itemsets by support and length for better analysis
frequent_itemsets.sort_values(by=['support', 'itemset_length'], ascending=[False, False], inplace = True)

# display top 10 frequent itemsets by support
print("\nTop` 10 Frequent Itemsets by Support:")
print(frequent_itemsets.head(10))

NameError: name 'TransactionEncoder' is not defined